# Исследование надежности заемщиков

****



## Открою таблицу и изучу общую информацию о данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\SubKi\Downloads\data_sets\data.csv")

**Выведу первые 20 строчек датафрейма `data` на экран.**

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведу основную информацию о датафрейме с помощью метода `info()`.**

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведу количество пропущенных значений для каждого столбца.**

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаю на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [6]:
for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (d) и есть пропуски в total_income 
    data.loc[(data['income_type'] == d) & (data['total_income'].isna()), 'total_income'] =  data.loc[(data['income_type'] == d), 'total_income'].median()

In [7]:
data['total_income'].isna().sum() # проверка на пустые значения в столбце тип доходов

0

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработать значения в этом столбце: заменю все отрицательные значения положительными с помощью метода `abs()`.**

In [8]:
data['days_employed'] = data['days_employed'].abs()

** Для каждого типа занятости выведу медианное значение трудового стажа `days_employed` в днях.**

In [9]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому пока оставлю их как есть.

**Выведу перечень уникальных значений столбца `children`.**

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалю строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [11]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведу перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполню пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [13]:
for d in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (d) и есть пропуски в days_employed 
    data.loc[(data['income_type'] == d) & (data['days_employed'].isna()), 'days_employed'] =  data.loc[(data['income_type'] == d), 'days_employed'].median()

**Убежусь, что все пропуски заполнены. Проверю себя и ещё раз выведу количество пропущенных значений для каждого столбца с помощью двух методов.**

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменю вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [15]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

**Обработаю неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведу их к нижнему регистру.**

In [16]:
data['education'] = data['education'].str.lower()

**Выведу на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалю их.**

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создам в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Использую собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [19]:
def categorize_income(income):
    if income <= 30000:
        return 'E'
    if income >= 30000 and income <= 50000:
        return 'D'
    if income >= 50001 and income <= 200000:
        return 'C'
    if income >= 200001 and income <= 1000000:
        return 'B'
    return 'A'


In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведу на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создам функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Использую собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучу данные в столбце `purpose` и определю, какие подстроки помогут правильно определить категорию.**

In [22]:
# создайте функцию categorize_purpose()
def categorize_purpose(purpose_category):
    if purpose_category.find('авто') >= 0:
        return 'операции с автомобилем'
    if purpose_category.find('жил') >= 0 or purpose_category.find('недвиж') >=0: 
        return 'операции с недвижимостью'
    if purpose_category.find('свадьб') >= 0:
        return 'проведение свадьбы'
    if purpose_category.find('образов') >= 0:
        return 'получение образования'


In [23]:
# примените функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данные

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
data.groupby('children', as_index=False)['debt'].agg(['count', 'mean'])

,children,count,mean
0,0,14091,0.075438
1,1,4808,0.092346
2,2,2052,0.094542
3,3,330,0.081818
4,4,41,0.097561
5,5,9,0.000000


- Среди всех групп наименьший показатель задолженостей у клиентов без детей: 7.54%. У клиентов с одним, двумя  и тремя детьми 9.23%, 9.45%  и 8.18% соответсвенно. Разница в 1% не несёт значительного различия по своевременности выплат из за количества детей. К тому же клиентов с детьми от 3х слишком мало для точных измерений.

Проведу анализ, как сильно влияет наличие детей к задержкам по выплате у клиентов.

In [25]:
data['have_children'] = data['children'] > 0

In [26]:
data.groupby('have_children', as_index=False)['debt'].agg(['count', 'mean','sum'])

,have_children,count,mean,sum
0,False,14091,0.075438,1063
1,True,7240,0.092403,669


- Доля задолженостей у клиентов без детей состовляет 7.54%.
Доля задолженостей у клиентов с детьми состовляет 9.24%

**Вывод:** Исходя из анализа данных, наличие детей в среднем уменьшает вероятность своевременной оплаты кредита.
Само же количество детей незначительно влияет на эту возможность.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data.groupby('family_status', as_index=False)['debt'].agg(['count', 'mean'])

,family_status,count,mean
0,Не женат / не замужем,2796,0.097639
1,в разводе,1189,0.070648
2,вдовец / вдова,951,0.066246
3,гражданский брак,4134,0.093130
4,женат / замужем,12261,0.075606


- Лучшие показатели по своевременной оплате у вдов и вдовцов: 6.6%.
Второе место делят те, кто разведён - 7% и те, кто состоит в браке - 7.5%.
На третьем месте клиенты в гражданском браке - 9.3% и те, кто не состоит в браке - 9.7%.
Разобью на две категории: клиенты с парой и холостые - чтобы понять, влияет ли само наличие пары на задолженности.

In [28]:
def categorize_family_status(row):
    try:
        if 'в разводе' in row or 'вдовец / вдова' in row or 'Не женат / не замужем' in row:
            return 'холостые'
        elif 'гражданский брак' in row or 'женат / замужем' in row:
            return 'есть пара'
    except:
        return 'нет категории'
data['common_status_family'] = data['family_status'].apply(categorize_family_status)

In [29]:
data.groupby('common_status_family', as_index=False)['debt'].agg(['count', 'mean'])

,common_status_family,count,mean
0,есть пара,16395,0.080024
1,холостые,4936,0.085089


- У клиентов в паре процент задолженостей составляет 8%, а у холостых 8.5%. Сильной разницы нет.

**Вывод:** По полученным данным, семейные люди более ответственные и лучше справляются с планированием бюджета для выплат, даже те, кто развёлся. Предположительно большая часть вдов и вдовцов пенсионеры, которые также состояли в браке (данных достаточно для проведения дополнительных иследований). Само наличие пары не сильно влияет на своевременную оплату долга (разница пол процента).
Между семейным положением и возвратом кредита в срок есть зависимость.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
data.groupby('total_income_category', as_index=False)['debt'].agg(['count', 'mean'])

,total_income_category,count,mean
0,A,25,0.080000
1,B,5014,0.070602
2,C,15921,0.084982
3,D,349,0.060172
4,E,22,0.090909


- В категориях 'А','D','Е' недостаточно большие выборки для уверенного расчёта. Тем не менее в категории зарплаты 'D'(30 001 – 50 000) наименьший уровень просрочивших выплату (6%). В категории 'B'(200 001 – 1 000 000) и 'C'(50 001 – 200 000) уже хорошая выборка. Чаще расчитываются по долгам люди в категории 'B'(7%), а в категории 'C' уже 8.4% клиентов с задолженностью.

**Вывод:** Между классом 'B' и 'C' есть зависимость от уровня дохода, в других группах велика вероятность погрешности в связи c малой выборкой. У людей с более высоким доходом возможность вернуть долг своевременно выше.

### Как разные цели кредита влияют на его возврат в срок?

In [31]:
data.groupby('purpose_category', as_index=False)['debt'].agg(['count', 'mean'])

,purpose_category,count,mean
0,операции с автомобилем,4279,0.093480
1,операции с недвижимостью,10751,0.072551
2,получение образования,3988,0.092528
3,проведение свадьбы,2313,0.079118


**Вывод:** Чаще всего вовремя отдают долг клиенты, взявшие кредит на недвижимость (7.2%) и на свадьбу (7.9%).
Хуже дела обстоят у тех, кто брал кредит на автомобиль (9.3%) и на образование(9.2%).

## Выводы

1. Исходя из анализа данных, наличие детей в среднем уменьшает вероятность своевременной оплаты кредита. Само же количество детей незначительно влияет на эту возможность.
2. По полученным данным, семейные люди более финансово граммотные и лучше справляются с планированием бюджета для выплат, даже те, кто развёлся. Предположительно большая часть вдов и вдовцов пенсионеры, которые также состояли в браке (данных достаточно для проведения дополнительных иследований). Само наличие пары не сильно влияет на своевременную оплату долга (разница пол процента).
Между семейным положением и возвратом кредита в срок есть зависимость.
3. Между классом 'B'(от 200 тыс.. до 1 млн.) и 'C'(от 50 тыс. до 200 тыс.) есть зависимость от уровня дохода, в других группах велика вероятность погрешности в связи c малой выборкой. У людей с более высоким доходом возможность вернуть долг своевременно выше.
4. Чаще всего вовремя отдают долг клиенты, взявшие кредит на недвижимость (7.2%) и на свадьбу (7.9%). Хуже дела обстоят у тех, кто брал кредит на автомобиль (9.3%) и на образование(9.2%)
5. По полученным данным, с наибольшей веротяность кредит вернут клиенты, состоящие в браке, без детей, с доходом от 200 тыс. до 1 млн., взявшие кредит на недвижимость. И с наименьшей вероятностью вовремя погасят задолженность клиенты, не состоящие в браке, с детьми, с доходом 50 тыс. – 200 тыс., взявшие кредит на автомобиль.